In [16]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
chat=ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

t=PromptTemplate(
    template="What is the capital of {country}",
    input_variables=["country"],

)
t.format(country="France")

'What is the capital of France'

***FewShotPromptTemplate***

In [6]:
examples=[
    {
        "question":"What do you know about France?",
        "answer":"""
        Here is what I know:
        Capital: Paris
        Language: Frence
        Food: Wine and Cheese
        Currency:Euro
        """,
    },
    {
        "question":"What do you know about Italy?",
        "answer":"""
        I know this:
        Capital:Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question":"What do you know about Greece?",
        "answer":"""
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

chat.predict("what do you know about France?")

France is a country located in Western Europe. It is known for its rich history, culture, and cuisine. The capital city is Paris, which is famous for landmarks such as the Eiffel Tower and the Louvre Museum. France is also known for its wine production, fashion industry, and art scene. The country has a diverse landscape, including mountains, beaches, and countryside. French is the official language, and the currency is the Euro. France is a member of the European Union and is one of the most visited countries in the world.

'France is a country located in Western Europe. It is known for its rich history, culture, and cuisine. The capital city is Paris, which is famous for landmarks such as the Eiffel Tower and the Louvre Museum. France is also known for its wine production, fashion industry, and art scene. The country has a diverse landscape, including mountains, beaches, and countryside. French is the official language, and the currency is the Euro. France is a member of the European Union and is one of the most visited countries in the world.'

In [13]:
example_template="""
    Human:{question}
    AI:{answer}
"""

example_prompt=PromptTemplate.from_template(example_template)

prompt=FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country="Germany")

chain=prompt|chat
chain.invoke({
    "country":"Korea"
})

AI: 
I know the following:
Capital: Seoul
Language: Korean
Food: Kimchi and Bibimbap
Currency: South Korean Won

AIMessageChunk(content='AI: \nI know the following:\nCapital: Seoul\nLanguage: Korean\nFood: Kimchi and Bibimbap\nCurrency: South Korean Won')

***FewShotChatMessagePromptTemplate***

In [24]:
examples=[
    {
        "country":"France",
        "answer":"""
        Here is what I know:
        Capital: Paris
        Language: Frence
        Food: Wine and Cheese
        Currency:Euro
        """,
    },
    {
        "country":"Italy",
        "answer":"""
        I know this:
        Capital:Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country":"Greece",
        "answer":"""
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [26]:
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate
example_prompt=ChatPromptTemplate.from_messages([
    ("human","What do you know about {country}"),
    ("ai","{answer}")
])

example_prompt=FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)
final_prompt=ChatPromptTemplate.from_messages(
    [
    ("system","You are a geography expert, you give short answers"),
    example_prompt,
    ("human","What do you know about {country}?"),
    ]
)

chain=final_prompt|chat
chain.invoke({"country": "Korea"})


        I know this:
        There are two Koreas - North Korea and South Korea
        Capital of South Korea: Seoul
        Language: Korean
        Food: Kimchi and Bibimbap
        Currency: South Korean Won

AIMessageChunk(content='\n        I know this:\n        There are two Koreas - North Korea and South Korea\n        Capital of South Korea: Seoul\n        Language: Korean\n        Food: Kimchi and Bibimbap\n        Currency: South Korean Won')

***LengBasedExampleSelector***

In [28]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector import LengthBasedExampleSelector



In [30]:
examples=[
    {
        "question":"What do you know about France?",
        "answer":"""
        Here is what I know:
        Capital: Paris
        Language: Frence
        Food: Wine and Cheese
        Currency:Euro
        """,
    },
    {
        "question":"What do you know about Italy?",
        "answer":"""
        I know this:
        Capital:Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question":"What do you know about Greece?",
        "answer":"""
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


In [38]:
from langchain.prompts.example_selector.base import BaseExampleSelector

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self,examples):
        self.examples=examples
    def add_example(self,example):
        self.examples.append(example)
    def select_examples(self,input_variables):
        from random import choice
        return [choice(self.examples)]


In [42]:
example_template="""
    Human:{question}
    AI:{answer}
"""

example_prompt=PromptTemplate.from_template(example_template)

# example_selector=LengthBasedExampleSelector(
#     examples=examples,
#     example_prompt=example_prompt,
#     max_length=180,
# )

example_selector=RandomExampleSelector(
    examples=examples,
)

prompt=FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Germany")

'\n    Human:What do you know about Greece?\n    AI:\n        I know this:\n        Capital: Athens\n        Language: Greek\n        Food: Souvlaki and Feta Cheese\n        Currency: Euro\n        \n\n\nHuman: What do you know about Germany?'

**How to load prompt templates from DISK**

In [46]:
from langchain.prompts import load_prompt

prompt=load_prompt("./prompt.json")
prompt.format(country="Germany")

'What is the capital of Germany'

In [49]:
from langchain.prompts import load_prompt
prompt=load_prompt("./prompt.yaml")
prompt.format(country="Korea")

'What is the capital of Korea'

**How to compose prompts?**

In [50]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.pipeline import PipelinePromptTemplate #많은 프롬프트들을 하나로 묶어주는 역할


chat=ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}

    {example}

    {start}
"""
)

prompt=[
    ("intro",intro),
    ("example",example),
    ("start",start),
]

full_prompt=PipelinePromptTemplate(final_prompt=final,pipeline_prompts=prompt)

full_prompt.format(
    character="Programmer",
    example_question="What is your role?",
    example_answer="I am an AI engineer",
    question="What is your specific research field?"
)

'\n    \n    You are a role playing assistant.\n    And you are impersonating a Programmer\n\n\n    \n    This is an example of how you talk:\n\n    Human: What is your role?\n    You: I am an AI engineer\n\n\n    \n    Start now!\n\n    Human: What is your specific research field?\n    You:\n\n'

In [51]:
chain=full_prompt|chat
chain.invoke({
    "character":"Programmer",
    "example_question":"What is your role?",
    "example_answer":"I am an AI engineer",
    "question":"What is your specific research field?"
})

I specialize in machine learning and natural language processing. My research focuses on developing algorithms and models that can understand and generate human language.

AIMessageChunk(content='I specialize in machine learning and natural language processing. My research focuses on developing algorithms and models that can understand and generate human language.')

**Caching**

In [53]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache,set_debug
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())

chat=ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

chat.predict("Could you explain me about CLIP, which is a vision-language Model?")

'CLIP (Contrastive Language-Image Pre-training) is a vision-language model developed by OpenAI that is capable of understanding and generating text about images in a way that is similar to how humans do. The model is trained on a large dataset of images and their corresponding text descriptions, allowing it to learn the relationship between visual and textual information.\n\nCLIP is unique in that it does not require any explicit alignment between images and text during training. Instead, it learns to associate images and text by maximizing the similarity between them in a process known as contrastive learning. This allows the model to generalize to new images and text descriptions that it has not seen before.\n\nOne of the key advantages of CLIP is its ability to perform a wide range of vision-language tasks, such as image classification, object detection, and image captioning, without the need for task-specific training. This makes it a versatile and powerful tool for a variety of ap

In [54]:
chat.predict("Could you explain me about CLIP, which is a vision-language Model?")

'CLIP (Contrastive Language-Image Pre-training) is a vision-language model developed by OpenAI that is capable of understanding and generating text about images in a way that is similar to how humans do. The model is trained on a large dataset of images and their corresponding text descriptions, allowing it to learn the relationship between visual and textual information.\n\nCLIP is unique in that it does not require any explicit alignment between images and text during training. Instead, it learns to associate images and text by maximizing the similarity between them in a process known as contrastive learning. This allows the model to generalize to new images and text descriptions that it has not seen before.\n\nOne of the key advantages of CLIP is its ability to perform a wide range of vision-language tasks, such as image classification, object detection, and image captioning, without the need for task-specific training. This makes it a versatile and powerful tool for a variety of ap

In [55]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache,set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(InMemoryCache())
set_debug(True)

chat=ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

chat.predict("Could you explain me about CLIP, which is a vision-language Model?")

Error in ConsoleCallbackHandler.on_llm_start callback: 1 validation error for Run
name
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type
Error in ConsoleCallbackHandler.on_llm_end callback: TracerException('No LLM Run found to be traced for 2a185ed4-811f-4a3f-8c2b-06fd83f9e7b4')


'CLIP (Contrastive Language-Image Pre-training) is a vision-language model developed by OpenAI that is capable of understanding and generating text about images in a way that is similar to how humans do. It is a large-scale neural network that has been trained on a diverse range of images and text data from the internet.\n\nCLIP is unique in that it is trained to understand both images and text simultaneously, allowing it to perform a wide range of tasks such as image classification, image generation, and image-text matching. This means that it can generate accurate descriptions of images, classify images into different categories, and even generate images based on textual descriptions.\n\nOne of the key features of CLIP is its ability to understand and generate text in a way that is contextually relevant to the images it is analyzing. This allows it to perform tasks such as zero-shot image classification, where it can accurately classify images into categories that it has never seen b

**만약 cache를 메모리가 아닌 데이터베이스로 비휘발성으로 관리하고 싶다면?**

In [56]:
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(SQLiteCache("cache.db"))

chat=ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

chat.predict("Could you explain me about CLIP, which is a vision-language Model?")

Error in ConsoleCallbackHandler.on_llm_start callback: 1 validation error for Run
name
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type
Error in ConsoleCallbackHandler.on_llm_end callback: TracerException('No LLM Run found to be traced for f9e39f28-0923-4496-bb48-83c8d4945370')


'CLIP (Contrastive Language-Image Pre-training) is a vision-language model developed by OpenAI that is capable of understanding both images and text. It is trained on a large dataset of images and their corresponding captions, allowing it to learn to associate images with their descriptions.\n\nOne of the key features of CLIP is its ability to perform zero-shot learning, meaning it can recognize objects and concepts in images that it has never seen before by understanding the text associated with the image. This is achieved through a process called contrastive learning, where the model is trained to distinguish between correct image-text pairs and incorrect pairs.\n\nCLIP has shown impressive performance on a wide range of vision-language tasks, such as image classification, object detection, and image generation. It has also demonstrated strong generalization capabilities, outperforming other vision-language models on various benchmarks.\n\nOverall, CLIP represents a significant advan

**Serialization & Cost Trace**

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback


chat=ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)
with get_openai_callback() as usage:
    chat.invoke("What are the SOTA models for Computer Vision, and NLP?")
    print(usage)

Tokens Used: 169
	Prompt Tokens: 21
	Completion Tokens: 148
Successful Requests: 1
Total Cost (USD): $0.0003275


**Serialization**

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI

chat=OpenAI(
    model="gpt-3.5-turbo-16k",
    temperatue=0.1,
    max_tokens=450,
)
chat.save("model.json")

/Users/seungwookim/Desktop/FULLSTCK-GPT/env/lib/python3.10/site-packages/langchain/utils/utils.py:159: UserWarning: WARNING! temperatue is not default parameter.
                temperatue was transferred to model_kwargs.
                Please confirm that temperatue is what you intended.
  warnings.warn(


In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI
from langchain.llms.loading import load_llm

chat=load_llm("model.json")
chat

/Users/seungwookim/Desktop/FULLSTCK-GPT/env/lib/python3.10/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/seungwookim/Desktop/FULLSTCK-GPT/env/lib/python3.10/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


OpenAIChat(client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo-16k', model_kwargs={'temperature': 0.7, 'max_tokens': 450, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}, 'temperatue': 0.1})